# Athena API Test
In this notebook we will try and test our **Sandbox** environment created via Athenahealth API. 

## Scheduling Appointment
What we will test is the ```appointment``` endpoint. The usual work-flow that is required from **Athenahealth API** is the following: 

1. Use ```GET``` ```/departments``` to get a list of departments at a practice. While this information can be cached, be aware that it does change from time to time.

2. Use ```GET``` ```/providers``` to get a list of providers. Again, this is cachable for a period of time, but can change.

3. Use ```GET``` ```/patientappointmentreasons``` to narrow down a specific appointment reason. You can use reasontype (or ```/patientappointmentreasons/existingpatient``` or ```/patientappointmentreasons/newpatient```) to limit to the patient reasons.

4. Use ```GET``` ```/appointments/open``` with a reason ID, department and provider (and date range) to retrieve a set of open appointments.

### New Patients
5. ```POST``` to ```/patients``` to create a new patient and get a patient ID. You must supply first and last name, a DOB, department ID, and at least one patient phone number (home, mobile, work).

6.  ```PUT``` to ```/appointments/{appointmentid}``` to book a patient, which will require the patient ID (from step 5), a reason ID (asked from the patient using the list from step 3), and an appointment ID (found in step 4).

### Existing Patients
5.  Use a ```GET``` to ```/patients``` to find the patient ID. There are certain minimum search criteria that must be met to find a patient: first and last name, date of birth, and at least one other demographic field (phone number, SSN, etc.). Refer to the detailed documentation for /patients.

6.  ```PUT``` to ```/appointments/{appointmentid}``` to book a patient, which will require the patient ID (from step 5), a reason ID (asked from the patient using the list from step 3), and an appointment ID (found in step 4).

## API Keys & Secrets
The API Keys and Secrets within this notebook are only temporary for this notebook. Therefor you will not be able to run these cells and will need to create your own developer account at: https://www.athenahealth.com/developer-portal

In [8]:
# importing
import http.client
import base64
import json
import urllib.request
import urllib.parse
import urllib.error

# importing src athenahealthapi module - from MDP repository
import sys
sys.path.insert(0, '../src/')
import athenahealthapi
import datetime

In [9]:
# Keys and parameters
key = "wvwz87ytfsd7sjpcvw3eqqwt"
secret = "AVW4VXha2wb6qm8"
version = "preview1" # API version to access
practiceid = 195900 # Sandbox id - used to hook into the hosptial practice 

In [10]:
# Instantiating API with our parameters
api = athenahealthapi.APIConnection(version, key, secret, practiceid)

## Formatting Quick Database to better understand API Call 
We will make a quick db (df) using Pandas to mimic what the API will retrieve via ```GET```. We will only mimic making an appointment with **New Patients** and **Existing Patients**

This purpose of this is just to get a feel of the API and it's calls.

In [50]:
import pandas as pd

In [155]:
# Grabbing all our raw data - returned as JSON, Dictionaries
departments = api.GET('/departments')
providers = api.GET('/providers')

# Counts
department_count = departments['totalcount']
provider_count = len(providers['providers'])

In [85]:
# fist department
departments['departments'][0]

{'creditcardtypes': ['DS', 'MC', 'VI'],
 'timezoneoffset': -5,
 'singleappointmentcontractmax': '3000',
 'state': 'GA',
 'placeofservicefacility': False,
 'latitude': '34.25597',
 'departmentid': '1',
 'address': '8762 STONERIDGE CT',
 'placeofservicetypeid': '11',
 'longitude': '-85.17026',
 'clinicals': 'ON',
 'timezone': -4,
 'name': 'MAIN ST (HUB)',
 'patientdepartmentname': 'Rome Office',
 'chartsharinggroupid': '42',
 'placeofservicetypename': 'OFFICE',
 'zip': '30417',
 'timezonename': 'US/Eastern',
 'communicatorbrandid': '1',
 'medicationhistoryconsent': False,
 'ishospitaldepartment': False,
 'providergroupid': '1',
 'portalurl': 'www.kenneth.net',
 'city': 'CLAXTON',
 'servicedepartment': True,
 'oneyearcontractmax': '1500',
 'fax': '(888) 555 1832',
 'providergroupname': 'Downtown Health Group',
 'doesnotobservedst': False,
 'phone': '(555) 916-7897',
 'ecommercecreditcardtypes': ['DS', 'MC', 'VI']}

In [86]:
# first provider
providers['providers'][0]

{'firstname': 'Terry',
 'specialty': 'Internal Medicine',
 'schedulingname': 'tahmad0',
 'providertypeid': 'MD',
 'billable': False,
 'displayname': 'Terry Ahmad, MD',
 'ansinamecode': 'Allopathic & Osteopathic Physicians : Family Practice (207Q00000X)',
 'lastname': 'Ahmad',
 'providerid': 86,
 'supervisingproviderusername': 'btrout0',
 'supervisingproviderid': 1,
 'ansispecialtycode': '207Q00000X',
 'hideinportal': False,
 'entitytype': 'Person',
 'providertype': 'MD',
 'createencounteroncheckin': False}

In [156]:
# Departments DF creation
df = pd.DataFrame(departments['departments'])
columns_to_keep = ['departmentid', 'patientdepartmentname', 'providergroupid', 'providergroupname']

departments_df = df[columns_to_keep] # final df

del df 
del columns_to_keep 

# Providers DF creation
df = pd.DataFrame(providers['providers'])
columns_to_keep = ['firstname', 'providertypeid','displayname', 'specialty', 'providerid']

providers_df = df[columns_to_keep]

del df
del columns_to_keep

In [98]:
departments_df.head()

,departmentid,patientdepartmentname,providergroupid,providergroupname
0,1,Rome Office,1,Downtown Health Group
1,21,Cartersville Office,1,Downtown Health Group
2,62,Blackstone patient-facing name (athenaNet),2,Family Specialists
3,102,Cardiology Consults,1,Downtown Health Group
4,142,LANGLEY HOSPITAL-INPT,2,Family Specialists


In [99]:
providers_df.head()

,firstname,providertypeid,displayname,specialty,providerid
0,Terry,MD,"Terry Ahmad, MD",Internal Medicine,86
1,Shayna,MD,"Shayna Avallone, MD",Internal Medicine,117
2,Shayna,MD,"Shayna Avallone, MD",Internal Medicine,118
3,Ariane,"NP, S",NaN,General Practice,79
4,Sanjuanita,MD,"Sanjuanita Briganti, MD",General Surgery,75


# Helper Functions
Below are helper task functions which we can later us in our API

In [273]:
# help functions
def path_join(*args):
    return ''.join('/' + str(arg).strip('/') for arg in args if arg)

def get_patient_reasons(provider_id_, department_id_):
    """
    ARGS:
        provider_id: <int>
        department_id: <int>
    """
    input_parameters = {
        "providerid": provider_id_, 
        "departmentid": department_id_
    }
    
    par = api.GET('/patientappointmentreasons', parameters=input_parameters) 
    
    return par


def get_available_appointments(department_id_, par_, return_limit=30):
    """
    ARGS:
        department_id: <int>
        par_: <int> - patient reason id
        return_limit: <int> - how many appointments to return
    """
    # HARDCODED DATES
    today = datetime.date.today()
    next_year = today.replace(year = today.year + 1)
    dateformat = "%m/%d/%Y"
    
    input_parameters= {
        "departmentid": department_id_,
        "startdate": today.strftime(dateformat),
        "enddate": next_year.strftime(dateformat),
        "appointmenttypeid": par_,
        "limit": 1
    }
    # API CALL
    open_appointments = api.GET('/appointments/open', parameters=input_parameters)
    
    # grabbing up to limit
    appointments = open_appointments['appointments'][:return_limit]
    
    del open_appointments
    
    return appointments, open_appointments

def create_new_patient(last_name_, first_name_, department_id_, dob_,address_1_=None, address_2_=None, city_=None,email_=None, state_=None, zipcode_=None):
    """
    This function will create a new patient - will be called when boolean logic determines this is a new patient. This function will return a patient ID which will be fed into PUT to scheduling an appointment
    ARGS:
       last_name: <string> REQUIRED
       first_name: <string> REQUIRED
       address_1: <string> OPTIONAL
       address_2: <string> OPTIONAL
       city_: <string> OPTIONAL
       department_id_: <int> REQUIRED
       email_: <string> OPTIONAL
       state_: <string> OPTIONAL
       zip_: <string> OPTIONAL
       dob_: <string> REQUIRED
    """
    
    patient_information = {
        "lastname": last_name_,
        "firstname": first_name_,
        "address1": address_1_,
        "address2": address_2_,
        "city": city_,
        "departmentid": department_id_,
        "email": email_,
        "state": state_,
        "zip": zip_,
        "dob": dob_
    }
    
    # API Call
    new_patient = api.POST('/patients', patient_information)
    
    # Grabbing patient id
    new_patient_id = new_patient[0]['patientid']
    
    return new_patient_id


def book_appointment(appointment_id_, par_, department_id_, patient_id_):
    """
    This function will then book an appointment based on all the above parameters
    ARGS:
        appointment_id_: <open_appointments['appointmentid']>
        par_: <int> - appointment type id
        department_id_: <int>
        patient_id_: <int>
    """
    
    appointment_info = {
        "appointmentid": par_,
        "departmentid": department_id_,
        "patient_id": patient_id_
    }
    
    # API Call
    booked = api.PUT(path_join('/appointments', appointment_id_), appointment_info)
    
    return booked

def get_departments():
    """
    This function will get all departments with given practice id (insantiated earlier). This will only return department name and the associated id.
    
    RETURN:
        depertment_id: department_name
    """
    # API Call
    departments = api.GET('/departments')
    
    df = pd.DataFrame(departments['departments'])
    columns_to_keep = ['departmentid', 'patientdepartmentname']

    departments_df = df[columns_to_keep] # final df

    del df 
    del columns_to_keep 
    
    departments_df.rename(columns={'patientdepartmentname':'departments'}, inplace=True)
    departments_df.set_index('departmentid', inplace=True)
    
    df_ = departments_df.to_dict() # converting to dictionary to return as
    
    del departments_df
    
    return json.dumps(df_)

def get_all_providers():
    """
    This function will return all providers given the practiceid
    """
    # API Call
    providers = api.GET('/providers')
    
    # Providers DF creation
    df = pd.DataFrame(providers['providers'])
    columns_to_keep = ['firstname', 'providertypeid','displayname', 'specialty', 'providerid']

    # creating out df
    providers_df = df[columns_to_keep].copy()

    # trimming
    providers_df.drop_duplicates(inplace=True)

    del df
    del columns_to_keep
    
    # Creating list
    providers = list(providers_df.apply(lambda el: el.to_dict(), axis=1))
    
    response_providers = {
        "providers": providers
    }
    
    return json.dumps(response_providers)

def json_to_dict(json_res):
    """
    helper function to convert to dictionary for better modularity. Mainly will be used for notebooks
    """
    return json.loads(json_res)

# API Test
Below we will be putting everything together and test via this notebook. Next step will be to actually create a RESTful API with the functions above. 

We will simulate an actual call to the API step by step, view the response and analyze.

In [291]:
# Call Helper Functions - functions to help with calls
def department_name_id_mapping(name_, keys_, values_):
    """
    ARGS:
        name_: <string>: name of department
        keys_: <list, string>: department id values
        values: <list, string>: department name values
    """
    # check if list exist in values
    if name_ in values_:
        
        # Finding index
        index_ = values_.index(name_) # index within name list
        d_name_index = keys_[index_]
        
        # returning the index <string>
        return d_name_index
        
    else:
        return f"Name not found: Here is the list: {values_}"
    
    
def department_parser(departments_res_):
    """
    This function will take raw json from get_dictionary() response return a list of all keys and values along with the dictionary
    ARGS:
        departments_dict_: <dict>
    """
    # converting json to dict
    departments = json_to_dict(departments_res_)
    
    # grabbing keys and values
    department_keys = list(departments['departments'].keys())
    department_values = list(departments['departments'].values())
    
    return department_keys, department_values, departments

def provider_id_mapping(name_, all_providers_):
    """
    This function will first check if the provider name you selected exist. If it does, it will then find that specific provider_id and return it. 
    
    The all_providers will be raw_json as will be the case in the API as out response will be a string
    
    FIX: This may be a problem if there are many specialist with the same name, therefor we need to match min two values.
    
    ARGS:
        name_: <string> - name user selected 
        all_providers_: <JSON>
    """
    # Converting RAW JSON to dictionary
    all_providers = json_to_dict(all_providers_)
    
    # looking for name
    find_name = list(filter(lambda person: person['firstname'] == name_, all_providers['providers']))
    
    if find_name:
        return find_name[0]['providerid']
    else:
        return "Specialist not found"

## 1. ```GET``` ```/departments```
Grabbing the list of all departments. We are most interested in retrieving the ```department_name_id``` from our helper functions with the input ```department_name```

In [249]:
# Getting all departments
all_departments = get_departments()

# Getting our list values
department_keys, department_names, _ = department_parser(all_departments)
print(f'All names: \n\n{department_names}')

All names: 

['Rome Office', 'Cartersville Office', 'Blackstone patient-facing name (athenaNet)', 'Cardiology Consults', 'LANGLEY HOSPITAL-INPT', 'CENTRAL CITY EMER RM', 'Belfast Primary Care', 'Portal Test', "7 Hills' & Co. Department", 'CommOpsDevTest', 'Main Office', 'WEST HILL HOSPITAL']


In [253]:
# Picking name and getting id number
department_name = "Cardiology Consults" # parameter from client

# Grabbing id
department_name_id = department_name_id_mapping(department_name, department_keys, department_names)

print(f'Name: {department_name}')
print(f'id: {department_name_id}')

Name: Cardiology Consults
id: 102


## 2. ```GET``` ```/providers```
Now that we have the ```department_name_id``` from the API, we most know retrieve all the providers within this practice. Therefore we are interested in ```provider_id``` from the input ```provider_name```.

In [292]:
# Getting all providers within practice
all_providers = get_all_providers()

In [294]:
# Picking a name - this will be selected via front-end
provider_name = 'Terry'

# grabbing provider id
provider_id = provider_id_mapping(provider_name, all_providers)

print(f'Provider name: {provider_name}')
print(f'id: {provider_id}')

Provider name: Terry
id: 86
